In [ ]:
data_path = '../dataset'

In [ ]:
import os

In [ ]:
for path in os.listdir(data_path):
    if os.path.isdir(os.path.join(data_path, path)):
        print(os.path.join(data_path, path))

In [ ]:
train_data_path = os.path.join(data_path, 'train')
test_data_path = os.path.join(data_path, 'test')

In [ ]:
train_classes = dict()

for path in sorted(os.listdir(train_data_path)):
    if os.path.isdir(os.path.join(train_data_path, path)):
        train_classes.setdefault(len(train_classes), path)
        
train_classes

In [ ]:
test_classes = dict()

for path in sorted(os.listdir(test_data_path)):
    if os.path.isdir(os.path.join(test_data_path, path)):
        test_classes.setdefault(len(test_classes), path)
        
test_classes

In [7]:
from torchvision import transforms as T

In [8]:
transform = T.Compose([
    T.Resize((72,72)),
    T.CenterCrop(64),
    T.ToTensor(),
    T.Normalize((.5, .5, .5), (.5, .5, .5))
])

In [9]:
from torchvision.datasets import ImageFolder

In [10]:
train_dataset = ImageFolder(
    root=train_data_path,
    transform=transform
)

In [11]:
test_dataset = ImageFolder(
    root=test_data_path,
    transform=transform
)

In [12]:
len(train_dataset), len(test_dataset)

(750, 2500)

In [13]:
from torch.utils.data import DataLoader

In [14]:
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    num_workers=0,
    shuffle=True
)

In [15]:
test_loader = DataLoader(
    test_dataset,
    batch_size=16,
    num_workers=0,
    shuffle=True
)

In [16]:
len(train_loader), len(test_loader)

(47, 157)

In [17]:
import torch.nn as nn
import torch.nn.functional as F

In [18]:
class FoodNet(nn.Module):
    def __init__(self):
        super(FoodNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 13 * 13, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 13 * 13)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [19]:
foodnet = FoodNet()

In [20]:
import torch.optim as optim

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(foodnet.parameters(), lr=0.001, momentum=0.9)

In [22]:
for epoch in range(10):
    running_loss = .0
    for i, data in enumerate(train_loader, 1):
        inputs, targets = data
        
        optimizer.zero_grad()
        
        outputs = foodnet(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    print(f"Loss {running_loss/i}")

Loss 2.3057554782705103
Loss 2.3045493034606284
Loss 2.303611552461665
Loss 2.302692301729892
Loss 2.3018523277120386
Loss 2.300767629704577
Loss 2.299435666266908
Loss 2.2980760462740633
Loss 2.296545145359445
Loss 2.294378569785585


In [23]:
!python --version

Python 3.8.5


In [26]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'